## Notebook to aggregate the downstream task performances for each model
After running single model evaluation with `../scripts/single_model_evaluation.py` for all models and datasets with can gather the linear probe results and store the aggregated information in a single csv file. 

In [ ]:
from itertools import product

import pandas as pd

from clip_benchmark.analysis.utils import retrieve_performance
from constants import exclude_models, model_config_file, BASE_PATH_RESULTS
from helper import load_all_datasetnames_n_info, load_model_configs_and_allowed_models, pp_storing_path

#### Global variables

In [ ]:
ds_list_perf, ds_info = load_all_datasetnames_n_info('../scripts/webdatasets_w_in1k.txt', verbose=True)

results_root = BASE_PATH_RESULTS / 'linear_probe/single_model'

SAVE = False
storing_path = pp_storing_path(BASE_PATH_RESULTS / f'aggregated/single_model_performance', SAVE)

#### Load config

In [ ]:
model_configs, allowed_models = load_model_configs_and_allowed_models(
    path=model_config_file,
    exclude_models=exclude_models,
    exclude_alignment=True,
)

### Aggreggate downstream task performance for all combinations of (ds_list_perf, allowed_models)

In [ ]:
res = []
for ds, mid in product(ds_list_perf, allowed_models):
    performance = retrieve_performance(
        model_id=mid,
        dataset_id=ds,
        metric_column='test_lp_acc1',
        results_root=results_root,
        regularization="weight_decay",
        allow_db_results=False
    )
    res.append({
        'DS': ds,
        'Model': mid,
        'TestAcc': performance
    })
perf_res = pd.DataFrame(res)

In [ ]:
if SAVE:
    perf_res.to_csv(storing_path / f'all_ds.csv', index=False)